In [5]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, GRU, SimpleRNN, Embedding, Conv1D, MaxPooling1D, Masking
from keras import optimizers,regularizers
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [6]:
def parse_dataset(name):
    infile = open(name, 'r')
    lines = infile.readlines()
    data = []
    for line in lines:
        data.append([float(x) for x in line.split(',')])
    y = [x[-1] for x in data]
    data = [x[:-1] for x in data]
    return data, y

In [7]:
n_steps=20
X_train, y_train = parse_dataset('data/estimation_without_padding.csv')
X_test, y_test = parse_dataset('data/competition_without_padding.csv')

In [8]:
def reshape_data(data):
    mat = [[] for _ in range(len(data))]
    for j in range(len(data)):
        row = data[j]
        mylist=np.empty(len(row))
        for i in range(0,len(row),2):
            mylist[i]=row[i]
            mylist[i+1]=row[i+1]
        mylist = np.reshape(mylist, (mylist.shape[0]//2,2))
        mat[j] = mylist
    return mat

In [9]:
# X_train = reshape_data(X_train)
# X_test = reshape_data(X_test)
maxlen = max(len(X_train), len(X_test))
X_train = pad_sequences(X_train, padding='post', value=-100, dtype=float, maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', value=-100, dtype=float, maxlen=maxlen)

In [10]:
X_train[0]

array([   1. ,   -0.3,    2. , ..., -100. , -100. , -100. ])

In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]//2, 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]//2, 2))

In [12]:
X_train.shape, X_test.shape, X_train[0]

((1170, 599, 2), (1198, 599, 2), array([[   1. ,   -0.3],
        [   2. ,   -0.3],
        [   1. ,   -0.3],
        ...,
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ]]))

In [13]:
model = Sequential()
model.add(Masking(mask_value=-100.0, input_shape=(None, 2)))
model.add(LSTM(10, return_sequences=False, activation='relu', input_shape=(None, 2)))
# model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=20)

Train on 1170 samples, validate on 1198 samples
Epoch 1/20
1170/1170 [==============================] - 53s 46ms/step - loss: 1.5478 - acc: 0.3932 - val_loss: 1.0426 - val_acc: 0.4549
Epoch 2/20
1170/1170 [==============================] - 55s 47ms/step - loss: 0.9596 - acc: 0.4821 - val_loss: 0.9155 - val_acc: 0.5259
Epoch 3/20
1170/1170 [==============================] - 55s 47ms/step - loss: 0.8753 - acc: 0.5085 - val_loss: 0.8544 - val_acc: 0.5326
Epoch 4/20
1170/1170 [==============================] - 55s 47ms/step - loss: 0.8240 - acc: 0.5291 - val_loss: 0.8101 - val_acc: 0.5334
Epoch 5/20
1170/1170 [==============================] - 53s 45ms/step - loss: 0.7818 - acc: 0.5667 - val_loss: 0.7772 - val_acc: 0.5309
Epoch 6/20
1170/1170 [==============================] - 52s 44ms/step - loss: 0.7612 - acc: 0.5735 - val_loss: 0.7585 - val_acc: 0.5526
Epoch 7/20
1170/1170 [==============================] - 52s 44ms/step - loss: 0.7428 - acc: 0.5855 - val_loss: 0.7427 - val_acc: 0.5634


In [ ]:
# for seq, label in zip(X_train, y_train):
#     model.fit(np.array([seq]), [label], batch_size=1)